# Implementing Feedforward Neural Networks with TesorFlow - Keras

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/AnwarMirza/CPS2025/blob/main/FNNwithTFKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/AnwarMirza/CPS2025/blob/main/FNNwithTFKeras.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

<!-- 
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AnwarMirza/CPS2025/blob/main/FNNwithTFKeras.ipynb)
-->

In [1]:
import tensorflow as tf
from tensorflow import keras

# Print the versions of TensorFlow and Keras
print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

TensorFlow version: 2.18.0
Keras version: 3.7.0


## Building an Image Classifier Using the Sequential API

Load Fashion MNIST dataset

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

# print shapes of training and testing data
print(f'X_train_full.shape = {X_train_full.shape}, X_train_full.dtype = {X_train_full.dtype}')
print(f'X_test.shape = {X_test.shape}, X_test.dtype = {X_test.dtype}')

X_train_full.shape = (60000, 28, 28), X_train_full.dtype = uint8
X_test.shape = (10000, 28, 28), X_test.dtype = uint8


We create the Keras model using the sequential API. This is the simplest of the Keras models for neural networks and is made up of a single stack of layers connected sequentially.

Within this sequential model we have:
* One `Flatten` layer that converts each input image (of size $28\times 28$) into a 1D array: if it receives the input data `X`, it computes `X.reshape(-1, 1)`. 
    * This layer does some simple preprocessing, involving only the shape of the input instances.
    * It does not involve the batch size.
    * Another way to replace this `Flatten` layer is to use `keras.layers.InputLayer` as the first layer, setting the `input_shape=[28,28]`.
* One `Dense` hidden layer with 300 neurons: 
    * Each neuron in this layer is connected with each neuron in the previous (i.e., input) layer.
    * Each neuron in this layer also receives a connection from a unit bias (thus resulting into 300 bias weights).
    * Each neurn in this layer uses ReLU as the activation function.
* One `Dense` hidden layer with 100 neurons:
    * Each neuron is connected with each neuron in the previous (dense layer). Thus resulting into $300\times 100$ weights.
    * Each neuron also receives a connection from a unit bias (resulting into 100 bias weights).
    * ReLU is used by each neuron as an activation function.
* One `Dense` output layer with 10 neurons:
    * Each neuron is connected with every neuron in the previous layer (resulting into $100\times 10$ weights).
    * Each neuron also receives a connection from a unit bias (resulting into $10$ bias weights).
    * The activation function for each neuron is `softmax`.
    * The number of neurons in this layer is equal to the number of output classes (which is 10).

In [3]:
model = keras.models.Sequential([
    # keras.layers.Input(shape=[28, 28]),
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


> An alternate way to make this model is as follows:
> 
    > ```python
    > model = keras.models.Sequential()
    > model.add(keras.layers.Flatten(input_shape=[28, 28]))
    > model.add(keras.layers.Dense(300, activation='relu'))
    > model.add(keras.layers.Dense(100, activation='relu'))
    > model.add(keras.layers.Dense(10, activation='relu'))
    >```
> 
> In this case, after creating a sequential model, each layer is added to it in each preceeding line of the code.  
>
> Each layer has been added sequentially to the model object. The activation function used in the dense layers is ReLU. This can be specified like this as well:  
> ```python
> model.add(keras.layers.Dense(100, activation=activation.relu))
> ```
>
> This single line can also be written as  
>```python
> model.add(layers.Dense(100))
> model.add(layers.Activation(activation.relu))
>```
>
> Here the activation function for the layer has been specified separately in the layer addition layer right after specifying the `Dense` layer.
>

There are several activation functions available in Keras. The full list is available at [https://keras.io/activations](https://keras.io/activations).

<table cellspacing="0" cellpadding="0" border="0" width="60%" align="center">
  <tr>
    <th valign="middle" align="center" width="125" bgcolor="#DDDDDD">
    Using Examples of Code from the keras.io Site
    </th>
  </tr>
  <tr>
    <td style="line-height:1.6;padding: 5px 15px;">
      <div>Example of code from the keras.io site work as they are with tf.keras, however, the import statements need to be changed. For example, lets have the following code from keras.io:</div>
      <pre>
        from keras.layers import Dense
        output_layer = Dense(10)</pre>
      <div>We must change it to</div>
      <pre>
        from tensorflow.keras.layers import Dense
        output_layer = Dense(10)</pre>
      <div>Or, we can simply use the full paths,</div>
      <pre>
        from tensorflow import keras
        output_layer = keras.layers.Dense(10)</pre>
      <div></div>
    </td>
  </tr>
</table>

In [4]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 300)            │       235,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 266,610 (1.02 MB)

 Trainable params: 266,610 (1.02 MB)

 Non-trainable params: 0 (0.00 B)